In [4]:
# rise time function
riseTime <- function( x_data, y_data, start_index, stop_index=start_index+as.integer(blockDuration/3) ) {
    ymax <- max(y_data[start_index:stop_index])
    ymax_index <- which( y_data == ymax )
    if ( length(ymax_index) > 1 ) {
        ttp <- ymax_index[1] - start_index
    } else {
        ttp <- ymax_index - start_index
    }
    start_offset <- 80 # should the baseline be hard-coded like this?
    if ( start_index > start_offset ) {
        baseline <- y_data[(start_index):start_index]
    } else {
        baseline <- y_data[(start_index-start_offset):start_index] 
    }
    baseline_value <- mean(baseline)
    peak_amplitude <- ymax - baseline_value
    if ( peak_amplitude < 0 ) {
        # no signal! return "NA"
        rise_time <- NA
    } else {
        rise_time <- peak_amplitude / ttp 
    }
    return( rise_time )
}

# support function to get block start locations
# for emo fear only
getBlockStarts <- function( glm1 , glm2 ) {  
    # generate x axis
    unit_time <- 1:length( glm1 )
    # find first block
    sum_glm <- fixglm_fear( glm1 + glm2 )
    N <- length(sum_glm)
    blockStartIndices <- array(0, dim=6)
    c <- 1
    for ( i in 10:(N-1) ) {    # leave a bit or wriggle room at the start
        if ( sum_glm[i] == 0 && sum_glm[i+1] > 0) {
            # found a block start location
            blockStartIndices[c] <- i+1
            c <- c+1
        }
    }
    if ( c != 7 ) {
        print("Error: GetBlockStarts:")
        print("Error in finding block start!")
        print("Expected 6 block start locations")
        print(paste0("Found ", c-1))
        return(list("blockStartIndices"=rep(-1, 6), "fors"=rep(-1, 6)))
    }
    fors <- array(0, dim=6)
    offset <- 10
    for ( j in 1:6 ) {
        if ( glm1[blockStartIndices[j]+offset] > 0) {
            fors[j] <- 1 #feel
        } else {
            fors[j] <- 2 #suppress
        }
    }
    return( list("blockStartIndices"=blockStartIndices, "fors"=fors) )
}

In [5]:
# loading other support functions
source('fnirs_rfunctions.R')

# testing EDA preprocessing
# load random datasets for testing
data1 <- importData('Fatigue_files_for_Yanlu_180323/BN-9457/BN-9457_emo_fear_pre.txt') #fear
data2 <- importData('Fatigue_files_for_Yanlu_180323/BJ-4714/BJ-4714_stroop_post.txt') #stroop
data3 <- importData('Fatigue_files_for_Yanlu_180323/LA-8900/LA-8900_VFT_post.txt') #vft

In [12]:
# taking a look at emo fear
par(mfrow=c(3,1))
glm1 <- fixglm_fear( data1$glm1 )
glm2 <- fixglm_fear( data1$glm2 )
blockIdx <- getBlockStarts( glm1 , glm2 )
blockIdx
unit_time <- (1:length(glm1))/8
rt1 <- riseTime(unit_time, data1$eda, blockIdx$blockStartIndices[1])

$blockStartIndices
[1]  272  832 1392 1952 2512 3072

$fors
[1] 2 1 2 1 1 2